# nominal regression model

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import sklearn
from sklearn.model_selection import train_test_split
from statsmodels import discrete
from statsmodels.discrete import discrete_model
from statsmodels.discrete.discrete_model import MNLogit


In [2]:
df = pd.read_excel('CDAC_DataBook.xlsx', sheet_name='nominal')


In [3]:
df.head()

,ses,write,math,prog
0,1,35,41,1
1,2,33,41,2
2,3,39,44,3
3,1,37,42,1
4,2,31,40,2


In [4]:
df = df[['ses','math','prog']]


In [5]:
df.head()

,ses,math,prog
0,1,41,1
1,2,41,2
2,3,44,3
3,1,42,1
4,2,40,2


In [6]:
ses_dummy = pd.get_dummies(df['ses'],drop_first=True)


In [7]:
df = df.drop('ses', axis=1)


In [8]:
df = pd.concat([df,ses_dummy], axis=1)


In [9]:
df.head()

,math,prog,2,3
0,41,1,0,0
1,41,2,1,0
2,44,3,0,1
3,42,1,0,0
4,40,2,1,0


In [10]:
x_train = df.drop('prog',axis=1)


In [11]:
y_train=df['prog']


In [12]:
x_train.head()

,math,2,3
0,41,0,0
1,41,1,0
2,44,0,1
3,42,0,0
4,40,1,0


In [13]:
y_train.head()

0    1
1    2
2    3
3    1
4    2
Name: prog, dtype: int64

In [ ]:
mod1 = sm.MNLogit(y_train,x_train).fit()


In [15]:
print(mod1.summary())


                          MNLogit Regression Results                          
Dep. Variable:                   prog   No. Observations:                  200
Model:                        MNLogit   Df Residuals:                      194
Method:                           MLE   Df Model:                            4
Date:                Mon, 26 Dec 2022   Pseudo R-squ.:                  0.2395
Time:                        10:29:51   Log-Likelihood:                -157.82
converged:                       True   LL-Null:                       -207.52
Covariance Type:            nonrobust   LLR p-value:                 1.332e-20
    prog=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
math          -0.0182      0.007     -2.630      0.009      -0.032      -0.005
2              3.1067      0.569      5.464      0.000       1.992       4.221
3              3.2378      1.126      2.876      0.0

In [18]:
#Ho: choice between prog1 and prog2 does not depend on the math score
#Ho: choice between prog1 and prog2 does not depend on whether you are from ses=1 (low) or ses=2 (middle)
#Ho: choice between prog1 and prog2 does not depend on whether you are from ses=1 (low) or ses=3 (high)

In [19]:
#choice b/w prog1 and prog2 does not depend on math score
#Ho is rejected
#choice b/w prog1 and prog2 depends on math score


In [20]:
mydata = pd.DataFrame([[60,0,0],[60,1,0],[60,0,1]], columns=['math','2','3'])


In [21]:
pred = mod1.predict(mydata)


In [23]:
pred


,0,1,2
0,0.639683,0.214742,0.145574
1,0.072404,0.543186,0.384410
2,0.018488,0.158123,0.823388


# ordinal logistic regression

In [24]:
#response in the ordinal scale .diff categories will have a logical order

In [26]:
import sys

In [32]:
sys.path.append(r'C:\Users\ZS283MZ\CDAC-Advanced_Statistical_Analysis\mord-0.6')


In [33]:
import mord

In [34]:
df = pd.read_excel('CDAC_DataBook.xlsx', sheet_name='ordinal')

In [35]:
df.head()

,Survival,Region,ToxicLevel
0,1,1,62.0
1,1,2,46.0
2,2,1,48.5
3,3,2,32.0
4,2,1,63.5


In [36]:
reg_dummy = pd.get_dummies(df['Region'], drop_first=True)


In [37]:
df = df.drop('Region', axis=1)


In [38]:
df = pd.concat([df,reg_dummy], axis=1)


In [39]:
x_train, x_test, y_train, y_test = train_test_split(df.drop('Survival', axis=1), df['Survival'], test_size=0.20)


In [40]:
from mord import LogisticAT


In [41]:
mod2 = LogisticAT().fit(x_train, y_train)


In [42]:
y_pred = mod2.predict(x_test)


In [43]:
from sklearn.metrics import confusion_matrix


In [45]:
print(confusion_matrix(y_test, y_pred))


[[0 4 0]
 [1 6 0]
 [1 3 0]]


In [46]:
from sklearn.metrics import classification_report


In [48]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           1       0.00      0.00      0.00         4
           2       0.46      0.86      0.60         7
           3       0.00      0.00      0.00         4

    accuracy                           0.40        15
   macro avg       0.15      0.29      0.20        15
weighted avg       0.22      0.40      0.28        15



C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [50]:
mydata = pd.DataFrame([[30,1],[40,1],[50,1],[70,1]], columns=['ToxicLevel','2'])



In [51]:
pred = mod2.predict(mydata)


In [52]:
pred

array([2, 2, 2, 1])

In [53]:
mydata.head()

,ToxicLevel,2
0,30,1
1,40,1
2,50,1
3,70,1


# count models


In [54]:
#response is a discrete data - response is in counts 
#poisson ,negative_binomial
#poisson- low varation in the response values
#negative binomial - high variation in the response values

In [55]:
import statsmodels.formula.api as smf
from statsmodels.discrete.discrete_model import Poisson as psn


In [56]:
df = pd.read_excel('CDAC_DataBook.xlsx', sheet_name='poisson')


In [57]:
df.head()

,num_awards,prog,math
0,0,3,41
1,0,1,41
2,0,3,44
3,0,3,42
4,0,3,40


In [58]:
prog_dummy = pd.get_dummies(df['prog'], drop_first=True)


In [59]:
df = df.drop('prog', axis=1)



In [60]:
df = pd.concat([df,prog_dummy], axis=1)


In [61]:
df.head()

,num_awards,math,2,3
0,0,41,0,1
1,0,41,0,0
2,0,44,0,1
3,0,42,0,1
4,0,40,0,1


In [62]:
x_train, x_test, y_train, y_test = train_test_split(df.drop('num_awards',axis=1),df['num_awards'], test_size=0.2)


In [63]:
x_train = sm.add_constant(x_train)


In [64]:
df_train = pd.concat([x_train,y_train], axis=1)


In [65]:
df_train.columns = ['const','math','prog2','prog3', 'num_awards']


In [66]:
mod3 = psn.from_formula('num_awards ~ math+prog2+prog3', data=df_train).fit()


Optimization terminated successfully.
         Current function value: 0.870079
         Iterations 7


In [68]:
print(mod3.summary())

                          Poisson Regression Results                          
Dep. Variable:             num_awards   No. Observations:                  160
Model:                        Poisson   Df Residuals:                      156
Method:                           MLE   Df Model:                            3
Date:                Mon, 26 Dec 2022   Pseudo R-squ.:                  0.2315
Time:                        11:47:32   Log-Likelihood:                -139.21
converged:                       True   LL-Null:                       -181.16
Covariance Type:            nonrobust   LLR p-value:                 4.492e-18
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -5.3654      0.741     -7.242      0.000      -6.818      -3.913
math           0.0727      0.012      5.942      0.000       0.049       0.097
prog2          0.9654      0.415      2.326      0.0

In [69]:
#Ho: num_awards does not depend on math score


In [70]:
#Ho: num_awards does not depend on math score
#Ho: num_awards do not change if we change our course from prog1 to prog2
#Ho: num_awards do not change if we change our course from prog1 to prog3


# negative regression

In [71]:
df = pd.read_excel('CDAC_DataBook.xlsx', sheet_name='neg_bin')

In [72]:
df.head()

,math,prog,daysabs
0,63,Academic,4
1,27,Academic,4
2,20,Academic,2
3,16,Academic,3
4,2,Academic,3


In [73]:
import statsmodels.formula.api as smf
from statsmodels.discrete.discrete_model import NegativeBinomial as ngb


In [74]:
prog_dummy = pd.get_dummies(df['prog'], drop_first = True)


In [75]:
df = df.drop('prog', axis=1)


In [76]:
df = pd.concat([df,prog_dummy], axis=1)


In [77]:
df.head()


,math,daysabs,General,Vocational
0,63,4,0,0
1,27,4,0,0
2,20,2,0,0
3,16,3,0,0
4,2,3,0,0


In [78]:
#academic course taken as ref event

In [79]:
x_train, x_test, y_train, y_test = train_test_split(df.drop('daysabs', axis=1), df['daysabs'], test_size=0.2)


In [80]:
x_train = sm.add_constant(x_train)


In [81]:
df_train = pd.concat([x_train, y_train], axis=1)


In [82]:
mod4 = ngb.from_formula("daysabs ~ math+General+Vocational", data=df_train).fit()


Optimization terminated successfully.
         Current function value: 2.836099
         Iterations: 21
         Function evaluations: 27
         Gradient evaluations: 27


In [83]:
print(mod4.summary())

                     NegativeBinomial Regression Results                      
Dep. Variable:                daysabs   No. Observations:                  251
Model:               NegativeBinomial   Df Residuals:                      247
Method:                           MLE   Df Model:                            3
Date:                Mon, 26 Dec 2022   Pseudo R-squ.:                 0.03026
Time:                        12:46:19   Log-Likelihood:                -711.86
converged:                       True   LL-Null:                       -734.07
Covariance Type:            nonrobust   LLR p-value:                 1.225e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.1697      0.146     14.887      0.000       1.884       2.455
math          -0.0053      0.003     -1.910      0.056      -0.011       0.000
General        0.4856      0.197      2.467      0.0

In [84]:
#Ho days of absence does not depend on math score
#Ho days of absence do not change if we go from academic course to general course
#Ho days of absence do not change if we go from academic course to vocational course
#Ho for math gets rejected (p-value<0.05)
#Concl: days of absence depends on the math score


In [85]:
#As the math score increases, the days of absence will decrease
#Assuming that thep-value for General is more than 0.05
#Ho for the General will NOT be rejected
#Conl: days of absence os the same for two students - one from Academic and the other from General (for the same math score)
#Going from academic to general, days of absence will increase but this is not statistically significant because p-value > 0.05


In [86]:
#Ho for the Vocational will be rejected
#Conl: days of absence is NOT the same for two students - one from Academic and the other from Vocational (for the same math score)
#Going from academic to vocational, days of absence will decrease and this is statistically significant because p-value < 0.05


In [87]:
x_test = sm.add_constant(x_test)


In [88]:
pred = round(mod4.predict(x_test))


In [89]:
pred

223    6.0
313    3.0
158    3.0
18     9.0
103    9.0
      ... 
200    3.0
69     7.0
175    3.0
203    3.0
291    7.0
Length: 63, dtype: float64

In [90]:
mydata = pd.DataFrame([[1,40,1,0],[1,60,1,0],[1,80,1,0]], columns=['const','math','General','Vocational'])


In [91]:
mod4.predict(mydata)

0    11.510443
1    10.352387
2     9.310842
dtype: float64

In [92]:
round(mod4.predict(mydata))

0    12.0
1    10.0
2     9.0
dtype: float64

In [93]:
mydata = pd.DataFrame([[1,50,0,0],[1,50,1,0],[1,50,0,1]], columns=['const','math','General','Vocational'])


In [94]:
round(mod4.predict(mydata))

0     7.0
1    11.0
2     3.0
dtype: float64

In [95]:
#multicollinearity
#predictors which are heavily correlated with each other
# X y
# x y
#y = 2x + 7+ var(0,0.5)


In [96]:
#Sir it means that for developing any model first we should consider multicollinearity and drop that particular things


In [97]:
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [98]:
df=pd.read_excel('CDAC_DataBook.xlsx',sheet_name='salaries')

In [99]:
df.head()

,rank,discipline,yrs_phd,yrs_service,gender,salary
0,Prof,B,19,18,Male,139750
1,Prof,B,20,16,Male,173200
2,AsstProf,B,4,3,Male,79750
3,Prof,B,45,39,Male,115000
4,Prof,B,40,41,Male,141500


In [100]:
mydf = df[['yrs_phd','yrs_service']]


In [101]:
mydf.head()

,yrs_phd,yrs_service
0,19,18
1,20,16
2,4,3
3,45,39
4,40,41


In [106]:
'''
VIF
variance inflation factor
<10  ---> no multi.
>10 --- column need to be dropeeed
higher the VIF, more is the contribution of that factor multicol
'''

'\nVIF\nvariance inflation factor\n<10  ---> no multi.\n>10 --- column need to be dropeeed\nhigher the VIF, more is the contribution of that factor multicol\n'

In [107]:
variance_inflation_factor(mydf.values,1)


15.367052069748848